<a href="https://colab.research.google.com/github/edinaldoab/challenge_insightplaces/blob/main/semana01_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 1
### Contextualização


Como pessoa engenheira de dados temos como uma de nossas responsabilidades a disponibilização dos dados para que os usuários interessados possam realizar suas análises.
<br>
<br>
Na imobiliária InsightPlaces temos um website que torna público os anúncios dos imóveis que pertencem ao portfólio da imobiliária. Este site é alimentado por uma API REST que consulta nosso sistema de banco de dados e retorna um arquivo JSON que é tratado pelo frontend e exibido na tela do usuário. A imobiliária utiliza como SGBD deste recurso (anúncios) o MongoDB que é um SGBD noSQL, ou seja, ele trabalha com dados não estruturados e suas collections são armazenadas no formato JSON.
<br>
<br>
Nosso cliente nessa demanda é a equipe de ciência de dados da InsightPlaces que precisa de um conjunto de dados para criar um modelo de machine learning para possibilitar a criação de um projeto da empresa que consiste em uma ferramenta de avaliação de imóveis usados.

(Reprodução: Alura)

# 1. Preparação do Ambiente

## a. Instalação das dependências

In [125]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [126]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

## b. Inicialização da SparkSession

In [127]:
import findspark
findspark.init()

In [128]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("challenge_imob") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

## c. Montagem do Drive

In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## d. Inicializando a  UI do Spark 

In [130]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [131]:
with open('/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dados/authTokenngrok.txt', 'r') as file:
  content = file.read()
  file.close()

my_authtoken = str(content)

In [132]:
get_ipython().system_raw('./ngrok authtoken ' + my_authtoken)
get_ipython().system_raw('./ngrok http 4050 &')

!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://331a-34-75-20-66.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://331a-34-75-20-66.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [133]:
spark

Carregamento dos dados:

# 2. Carregamento e tratamento dos dados

## a. Leitura do dataset

In [134]:
path = '/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dados/dataset_bruto.json'

data = spark.read.json(path)

In [135]:
data.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [136]:
data.columns

['anuncio', 'imagens', 'usuario']

In [137]:
data.count()

89083

Dicionário de Dados (disponibilizado pela Alura)

| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |





## b. Ciração do data frame com as informações de anúncio

O dataset original contem três blocos principais:
*   anuncio
*   imagens
*   usuario

Seguindo a demanda do time de Data Science, as análises e o tratamanto de informações devem se ater aos dados de anúncio. Sendo assim, cria-se o dataset `anuncio`.

In [138]:
anuncio = data.select('anuncio.*')

anuncio.show(10, truncate=False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+

## c. Filtragem da base de dados

Foi demandado também que as colunas `tipo_uso`, `tipo_unidade` e `tipo_anuncio` recebessem os seguintes filtros:


*   tipo_uso: somente Residencial
*   tipo_unidade: somente Apartamento
*   tipo_anuncio: somente Residencial



In [139]:
anuncio\
    .select('tipo_uso')\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [140]:
anuncio=anuncio.filter('tipo_uso=="Residencial"')

In [141]:
anuncio\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84541|
+-----------+-----+



Avaliando a coluna `tipo_unidade`:

In [142]:
anuncio\
    .select('tipo_unidade')\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7517|
| Apartamento|66797|
|        Casa|10227|
+------------+-----+



In [143]:
anuncio = anuncio.filter('tipo_unidade=="Apartamento"')

In [144]:
anuncio\
    .select('tipo_unidade')\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66797|
+------------+-----+



Avaliando a coluna `tipo_anuncio`:

In [145]:
anuncio\
    .select('tipo_anuncio')\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
|  Lançamento|  235|
+------------+-----+



In [146]:
anuncio = anuncio.filter('tipo_anuncio=="Usado"')

In [147]:
anuncio\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



In [148]:
anuncio.limit(10).show(truncate=False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |endereco                                                                                                               |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                    |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+--------------

## d. "desmembrando" as atribuições de listas em valores:

Olhando para o schema da base de dados e para a impressão do data frame, nota-se que alguns dados que deveriam ter atribuições numéricas como `andar` e `area_total`, por exemplo, estão disponibilizados como listas. Assim, torna-se necessária a conversão **lista para int** - que neste primeiro momento, constará somente da extração dos valores internos da lista.

In [149]:
from pyspark.sql import functions as f

Deve-se garantir que todas as listas contém somente um elemento (indíce 0):

In [150]:
anuncio\
    .select(f.size(f.col('quartos')).alias('quartos'))\
    .groupBy('quartos')\
    .count()\
    .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+



Os seguintes dados receberão esta tratativa:

In [151]:
caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [152]:
for col in caracteristicas:
    anuncio.select(f.size(f.col(col)).alias(col)).groupBy(col).count().show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



`suites` e `vagas` possuim atributos vazios, então, pode-se inferir que o imóvel não possui estas dependências. A `area_total` também possui atribuições vazias, mas neste caso, não há decisões a serem tomadas por não se conhecer a causa da falta de registros.

In [153]:
anuncio\
    .select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])\
    .show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [154]:
anuncio=anuncio.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])

In [155]:
anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [156]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

## e. Extração dos dados de localização



Para a futura implementação do modelo de ML, a equipe de DS, precisará somente das informações de **bairro** e **zona** que compõem os dados de localização. 

In [157]:
anuncio\
    .select('endereco.*')\
    .show()

+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|
+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43.634034|  BR|Estrada de Santa ...|Zona Oeste|
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.928108|-43.635375|  BR|Estrada de Santa ...|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948756|-43.582824|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|Rio de Janeiro|Rio de Janeiro|-22.888194|-43.629602|  BR|Estrada da Paciência|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948291|-43.582205|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|R

In [158]:
anuncio\
    .select('*', 'endereco.bairro', 'endereco.zona')\
    .drop('endereco')\
    .show(5)

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|   bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+---------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|Paciência|Zona Oeste|
|    1|        41|       41|        1|[Portaria 24h, Co...|18d22cbe-1b86-476...|      2|  null|

In [159]:
anuncio = anuncio\
    .select('*', 'endereco.bairro', 'endereco.zona')\
    .drop('endereco')

## f. Tratamento para a coluna de valores

Como se trata de um estudo sobre o preço de venda dos imóveis, o time de cientistas de dados solicitou apenas as informações do tipo `Venda`.

In [160]:
anuncio\
    .select('valores')\
    .show()

+--------------------+
|             valores|
+--------------------+
|[{245, null, Vend...|
|[{0, 0, Venda, 15...|
|[{0, 0, Venda, 20...|
|[{285, null, Vend...|
|[{245, null, Vend...|
|[{285, null, Vend...|
|[{250, null, Vend...|
|[{245, null, Vend...|
|[{245, null, Vend...|
|[{240, null, Vend...|
|[{0, 0, Venda, 15...|
|[{240, 0, Venda, ...|
|[{245, null, Vend...|
|[{290, null, Vend...|
|[{285, null, Vend...|
|[{0, 0, Venda, 30...|
|[{null, null, Ven...|
|[{280, 0, Venda, ...|
|[{280, null, Vend...|
|[{0, 0, Venda, 26...|
+--------------------+
only showing top 20 rows



In [161]:
anuncio\
    .select(f.explode('valores').alias('valores'))\
    .select('valores.*')\
    .filter('valores.tipo=="Venda"')\
    .count()

66562

In [162]:
anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .show(10, truncate=False)

+------------------------------------+----------+----+-----+-----+
|id                                  |condominio|iptu|tipo |valor|
+------------------------------------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |null|Venda|15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda|15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda|20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |null|Venda|20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |null|Venda|15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |null|Venda|20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |null|Venda|15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |null|Venda|15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |null|Venda|15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |null|Venda|17999|
+------------------------------------+----------+----+-----+-----+
only showing top 10 rows



In [163]:
valores=anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .filter('valores.tipo=="Venda"')

In [164]:
valores.show()

+--------------------+----------+----+-----+-----+
|                  id|condominio|iptu| tipo|valor|
+--------------------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a...|       245|null|Venda|15000|
|085bab2c-87ad-452...|         0|   0|Venda|15000|
|18d22cbe-1b86-476...|         0|   0|Venda|20000|
|bed8a354-9317-442...|       285|null|Venda|20000|
|12a13315-d67f-48f...|       245|null|Venda|15000|
|a2e6d7a5-0ff0-484...|       285|null|Venda|20000|
|a6e3173b-c950-4db...|       250|null|Venda|15000|
|2e6e5dfb-206c-496...|       245|null|Venda|15000|
|99f8d0f9-95a4-461...|       245|null|Venda|15000|
|b3f44c1a-2d50-4d5...|       240|null|Venda|17999|
|dc99d9e1-4c63-41f...|         0|   0|Venda|15000|
|aa3606d8-8bcd-45e...|       240|   0|Venda|19999|
|df80b0d5-677c-4be...|       245|null|Venda|19999|
|3d6c0218-9b7d-474...|       290|null|Venda|25000|
|304266cc-62d1-4ce...|       285|null|Venda|29999|
|111472a2-afa1-4a7...|         0|   0|Venda|30000|
|275c1589-6537-4bf...|      nul

## g. Consolidação do dataset

In [165]:
anuncio.join(valores, 'id', how='inner').count()

66562

In [166]:
anuncio.join(valores, 'id', how='inner').show(5)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+----------+----+-----+-------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|condominio|iptu| tipo|  valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+----------+----+-----+-------+
|006e2b19-f4de-450...|    0|       116|      116|        2|[Elevador, Academ...|      3|     1|       Usado| Apartamento|Residencial|   1|[{100, 100, Venda...|     Barra da Tijuca|Zona Oeste|       100| 100|Venda|3793260|
|011acce0-129f-4c2...|    0|       143|      143|        4|[Churrasqueira, P...|      4|     3|       Usado| Apa

In [167]:
anuncio.join(valores, 'id', how='inner').drop('valores').show(5)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|      zona|condominio|iptu| tipo|  valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-------+
|006e2b19-f4de-450...|    0|       116|      116|        2|[Elevador, Academ...|      3|     1|       Usado| Apartamento|Residencial|   1|     Barra da Tijuca|Zona Oeste|       100| 100|Venda|3793260|
|011acce0-129f-4c2...|    0|       143|      143|        4|[Churrasqueira, P...|      4|     3|       Usado| Apartamento|Residencial|   1|         Jacarepaguá|Zona Oeste|      3948|2000|Venda|1600

In [168]:
anuncio = anuncio.join(valores, 'id', how='inner').drop('valores')

# Disponibilização do dataset tratado no formato [PARQUET](https://parquet.apache.org/)

In [170]:
anuncio.write.parquet(
    path='/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dataset_parquet',
    mode='overwrite'
)